In [1]:
import numpy as np
import pandas as pd

### CVXPY used to formulate optimization problem
### GUROBI is a powerful commercial solver that is free for academics

In [2]:
import cvxpy as cp
from gurobipy import *

load in the csvs that will be used in the optimization problem

In [3]:
df_distance_matrix = pd.read_csv("bus-to-garage.csv")

In [4]:
# remove unnessecary rows
df_distance_matrix.drop(["route_id", "route_shor", "trip_id", "trip_heads", "trip_short", "direction_", "agency_id", 
         "headway_se", "O_TERMINAL", "LEN_KM", "STD_FARE", "route_time_s", "buses", "standard_buses", "InputID"], axis=1, inplace= True)
df_distance_matrix.head()

,Al Amal,Al Amiriyyah,Al Sawah,Al-Mostaqbal,Athar an Naby,Badr,Cairo,El Mizalat,El-Mounib,El-Obour,El-Teraa El-,Fateh,Fom El-Khalig,Gesr Al Suez,Giza,Helwan,Imbaba,Maadi,Nasr,Port Said
0,36602.947800,36737.333846,37163.014690,43985.500356,4706.540707,603.670463,46065.919154,31873.720265,5320.046517,18138.561511,33144.529165,27057.702470,11299.512117,50029.417353,9419.272326,43267.236166,23033.434126,44464.477634,26702.330924,34721.879973
1,44528.234853,18663.809347,20273.023103,39756.488007,23418.258851,24044.450263,52333.570568,9277.844746,24085.930348,34808.775219,11721.629290,21515.022370,14405.590660,34510.291069,20095.310912,65044.147038,3745.091512,50196.996348,21844.863828,14862.254772
2,42083.436805,14396.101415,13156.703940,23601.657350,46006.466889,49484.545737,44313.009097,24220.630471,52160.423693,47669.165355,21489.312651,24332.987592,39080.300243,3009.943377,50137.987970,76359.276547,36727.364040,42306.063913,24933.431742,18141.236707
3,28805.441163,35767.492237,35729.071827,38207.693264,4915.862212,8433.592869,38290.126023,33183.830915,13373.115033,10118.927145,33824.201628,23404.657510,13288.181302,47224.307354,17074.095465,37603.227509,27130.514073,36774.377493,22896.898963,34620.652102
4,37726.555214,23322.041100,21461.090044,17327.340552,49905.600623,54007.201501,37586.609003,33735.387270,57458.974555,48076.403022,30966.754132,27175.189755,44559.694231,8675.768832,56371.265294,74053.626919,45292.732334,35856.561548,27577.292323,27453.482739


Transform dataframe to a numpy array

In [5]:
np_distance_matrix = df_distance_matrix.to_numpy()
#show the array
np_distance_matrix

array([[36602.9478002 , 36737.333846  , 37163.01469   , ...,
        44464.4776338 , 26702.3309244 , 34721.8799728 ],
       [44528.2348526 , 18663.80934736, 20273.0231032 , ...,
        50196.9963482 , 21844.8638278 , 14862.2547721 ],
       [42083.436805  , 14396.10141506, 13156.70394008, ...,
        42306.063913  , 24933.4317424 , 18141.23670674],
       ...,
       [37266.6979358 , 15360.66036834, 16365.82700662, ...,
        42857.7408182 , 14586.80167694, 12660.65787474],
       [29677.5706122 , 22691.9183036 , 22853.268479  , ...,
        36389.2402602 , 12546.3269017 , 21365.7910304 ],
       [44528.2348526 , 18663.80934736, 20273.0231032 , ...,
        50196.9963482 , 21844.8638278 , 14862.2547721 ]])

check dimensions

In [6]:
np.shape(np_distance_matrix)

(340, 20)

The depots are in alphabetical order. The csv with depot capacities needs to have the depots in alphabetical order as well!

IMPORTING CSV WITH GARAGE CAPACITIES

In [7]:
df_capacities = pd.read_csv("Garage Capacities.csv")
list(df_capacities.columns) 

['Name', 'Capacity']

In [8]:
df_capacities.drop(["Name"], axis=1, inplace= True)
df_capacities.head()

,Capacity
0,85
1,225
2,140
3,200
4,225


convert to a numpy array

In [9]:
np_depot_capacities = df_capacities.to_numpy()
# show the output
np_depot_capacities

array([[ 85],
       [225],
       [140],
       [200],
       [225],
       [140],
       [100],
       [225],
       [250],
       [100],
       [170],
       [225],
       [100],
       [250],
       [ 70],
       [140],
       [225],
       [100],
       [250],
       [160]])

In [10]:
np.shape(np_depot_capacities)

(20, 1)

We need to change this so that the capacities are all in one row. This way it will match the distance matrix array and
we can sum over the columns

In [11]:
# use flatten to collapse array into one dimension
np_depot_capacities = np_depot_capacities.flatten()
np_depot_capacities

array([ 85, 225, 140, 200, 225, 140, 100, 225, 250, 100, 170, 225, 100,
       250,  70, 140, 225, 100, 250, 160])

We need another array for the number of buses on each trip. This is necessary for the depot capacity constraint

In [12]:
# read in the csv
df_buses = pd.read_csv("bus-to-garage.csv")
# keep only the buses column
df_buses = df_buses[["standard_buses"]]
df_buses.head()

,standard_buses
0,4.0
1,4.0
2,26.0
3,26.0
4,11.0


In [13]:
np_buses = df_buses.to_numpy()
np_buses = np_buses.flatten()
np_buses

array([ 4.,  4., 26., 26., 11., 11.,  8.,  8.,  5.,  5.,  6.,  6.,  5.,
        5.,  5.,  5.,  8.,  8.,  4.,  4., 14., 14.,  4.,  4., 74., 74.,
        8.,  8.,  3.,  3.,  9.,  9.,  3.,  3.,  2.,  2.,  7.,  7., 14.,
       14.,  4.,  4.,  5.,  5.,  3.,  3., 62.,  6.,  6.,  4.,  4.,  5.,
        5.,  3.,  3.,  3.,  3.,  6.,  6.,  8.,  8., 13., 13.,  5.,  5.,
       26., 26., 31.,  3.,  3.,  4.,  4.,  6.,  6., 10., 10.,  6.,  6.,
       27., 27.,  7.,  4.,  4., 33., 33.,  8.,  8.,  3.,  3., 11., 11.,
        3.,  3., 11., 11.,  8.,  8.,  4.,  4.,  5.,  5.,  5.,  5.,  9.,
        9.,  5.,  5.,  6.,  6.,  8.,  9.,  9.,  7.,  7.,  3.,  3.,  4.,
        4.,  5.,  5., 10., 10.,  3.,  3.,  4.,  4., 12., 12.,  6.,  6.,
        5.,  5., 23., 23.,  3.,  3.,  4.,  4., 11., 11.,  5.,  5., 15.,
       15.,  5.,  5.,  3.,  3.,  6.,  6.,  4.,  4.,  7.,  7.,  5.,  5.,
        2.,  2.,  8.,  8., 11., 11.,  4.,  4.,  4.,  4.,  4.,  4., 12.,
       12.,  4.,  4.,  4., 36., 36., 13., 13.,  7.,  7.,  5.,  5

In [14]:
# we need to repeat each row 20 times, where 20 is the number of depots
# np.tile 'tiles' the array 20 times (20 rows of 340 values)
# .T transposes the array so that it is 340 rows with each row having the number of buses 
# of the respective trip repeated 20 times
buses = np.tile(np_buses,(20,1)).T
buses
#np.shape(buses)    # (340, 20)

array([[ 4.,  4.,  4., ...,  4.,  4.,  4.],
       [ 4.,  4.,  4., ...,  4.,  4.,  4.],
       [26., 26., 26., ..., 26., 26., 26.],
       ...,
       [ 5.,  5.,  5., ...,  5.,  5.,  5.],
       [14., 14., 14., ..., 14., 14., 14.],
       [14., 14., 14., ..., 14., 14., 14.]])

# SETTING UP THE PROBLEM

## Problem is set up as an LP Relaxation Problem. This is because it cannot be solved as an IP (340*20 variables = 6800 : time taken is 2^3600)
## Branch and Bound is then used to find approximate integer values

### selection is a binary '0/1' matrix. 1 means bus is assigned to terminal. If binary then (I think) the problem is np complete and solver keeps running for hours, an alternative is to allow positive fractions

In [15]:
selection = cp.Variable(shape=buses.shape,boolean=True)
#try without boolean
#selection = cp.Variable(shape=buses.shape, nonneg=True)



## Constraints

### trip can only be assigned to one depot (Row sums need to equal 1)

In [16]:
assignment_constraint = cp.sum(selection,axis=1) == 1

### depots cannot exceed capacities. Column sums <= depot capacity

#### cp.multiply(selection, buses) gets the number of buses that will be assigned to a garage. We are treating this as an all or nothing assignment problem. This means that either all buses from the same route are assigned to a garage or non of them are (they are not split up between garages)

In [17]:
capacity_constraint = cp.sum(cp.multiply(selection, buses), axis=0) <= np_depot_capacities

#### Add a max distance constraint 
##### 20000 equivalent to 20km

In [18]:
distance_constraint = cp.sum(cp.multiply(selection, np_distance_matrix), axis=1) <= 50000

### Minimize dead mileage. Dead mileage is the product of the distance matrix and the (buses*selection)

In [19]:
cost = cp.sum(cp.multiply(cp.multiply(selection, buses), np_distance_matrix))

Check installed solvers

In [20]:
print (cp.installed_solvers())

['CPLEX', 'CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'GUROBI', 'OSQP', 'SCS']


### Run the optimization problem. MAJOR FLAW (SOME ROWS ARE NOT ASSIGNED AS ALL COLUMNS FOR THAT ROW ARE <0.5 AND SO ROUND TO 0. IS THIS A FLAW IN ECOS_BB?

In [21]:
constraints = [assignment_constraint, capacity_constraint]

assign_problem = cp.Problem(cp.Minimize(cost),constraints)


assign_problem.solve(solver= cp.GUROBI, verbose = True)

# use ECOS_BB for branch and bound (should return integer solutions: some rows sum to zero after rounding :( )
#assign_problem.solve(solver= cp.ECOS_BB)

# ECOS_BB with max iterations: used when selection variable is boolean (IP problem). Doesn't find a solution
#assign_problem.solve(solver= cp.ECOS_BB, mi_max_iters= 1000)



print(selection.value)
print('Problem status: {}'.format(assign_problem.status))

Using license file /Library/gurobi900/gurobi.lic
Academic license - for non-commercial use only
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter QCPDual to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 360 rows, 6800 columns and 13600 nonzeros
Model fingerprint: 0x1c878aa2
Variable types: 0 continuous, 6800 integer (6800 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [1e+03, 9e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 8.937586e+07
Presolve removed 0 rows and 2 columns
Presolve time: 0.03s
Presolved: 360 rows, 6798 columns, 13596 nonzeros
Variable types: 0 continuous, 6798 integer (6798 binary)

Root relaxation: objective 3.330038e+07, 448 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth I

Now save the output as a csv. We can use this to check that the assignment_constraint was not violated

In [22]:
# save before rounding 
np.savetxt("assignment.csv", selection.value, delimiter=",")

In [23]:
type(selection.value)

numpy.ndarray

In [24]:
## No need for this. GUROBI SOLVES IT USING BRANCH AND BOUND

In [25]:
# round numpy array
#selection.value = selection.value.round()
#np.savetxt("assignment.csv", selection.value, delimiter=",")

Let's check if buses were allocated correctly. This shows how many buses were allocated to each garage. We use this to check that the garage capacities were not exceeded (capacity_constraint not violated)

In [26]:
allocation = selection.value*buses
np.savetxt("allocation.csv", allocation, delimiter=",")

### Get dead mileage per trip

In [27]:
test = (selection.value*buses) * np_distance_matrix

np.savetxt("total_mileage_per_trip.csv", test, delimiter=",")

#### Dead Mileage per trip (not weighted by number of buses on each trip)

In [28]:
test2 = (selection.value * np_distance_matrix)

np.savetxt("mileage_per_trip.csv", test2, delimiter=",")